In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mne
from mne import io
from mne import read_epochs

In [2]:
url = './data/'

print("\n========= epoch_1 ==========")

# epoch_1
file_name_1 = 'g1dual1_epo.fif' # eg. 'g1dual1_epo.fif'
epoch_1 = read_epochs(url + file_name_1)
print("\n========= epoch_2 ==========")

# epoch_2
file_name_2 = 'g1solo1_epo.fif' # eg. 'g1dual2_epo.fif'
epoch_2 = read_epochs(url + file_name_2)


========= epoch_1 ==========
Reading /home/st122403/work/eeg/data/g1dual1_epo.fif ...
    Found the data of interest:
        t =     -99.61 ...     699.22 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
636 matching events found
No baseline correction applied
0 projection items activated

========= epoch_2 ==========
Reading /home/st122403/work/eeg/data/g1solo1_epo.fif ...
    Found the data of interest:
        t =     -99.61 ...     699.22 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated


In [3]:
X_1 = epoch_1.get_data()
y_train = np.array(epoch_1.events[:, -1], dtype = 'float32')
y_train[y_train == 1.0] = -1.0
y_train[y_train == 2.0] = 1.0

print(X_1.shape)
print(y_train.shape)
# print(y)

(636, 32, 410)
(636,)


In [4]:
X_2 = epoch_2.get_data()
y_test = np.array(epoch_2.events[:, -1], dtype = 'float32')

y_test[y_test == 1.0] = -1.0
y_test[y_test == 2.0] = 1.0

print(X_2.shape)
print(y_test.shape)
# print(y)

(240, 32, 410)
(240,)


In [5]:
X_train = X_1.reshape(X_1.shape[0], X_1.shape[1] * X_1.shape[2])
X_test = X_2.reshape(X_2.shape[0], X_2.shape[1] * X_2.shape[2])

print(X_train.shape)
print(X_test.shape)

(636, 13120)
(240, 13120)


In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from mne.preprocessing import Xdawn
from pyriemann.estimation import XdawnCovariances
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [7]:
def search_cv(x_train, y_train, x_test, y_test, model=SVC()):

    parameters = {'kernel': ('linear', 'rbf'), 'degree': [3, 4, 5], 'C': [0.8, 0.9, 1.0]}
    clf = GridSearchCV(model, param_grid=parameters)
    grid_search = clf.fit(x_train, y_train)

    print("Best score: %0.3f" % grid_search.best_score_)
    print(grid_search.best_estimator_)

    # best prarams
    print('best prarams:', clf.best_params_)

    print('-----grid search end------------')
    print('on all train set')
    scores = cross_val_score(grid_search.best_estimator_, x_train, y_train, cv=3, scoring='accuracy')
    print(scores.mean(), scores)
    print('on test set')
    scores = cross_val_score(grid_search.best_estimator_, x_test, y_test, cv=3, scoring='accuracy')
    print(scores.mean(), scores) 

In [8]:
search_cv(X_train, y_train, X_test, y_test)

Best score: 0.838
SVC()
best prarams: {'C': 1.0, 'degree': 3, 'kernel': 'rbf'}
-----grid search end------------
on all train set
0.8364779874213837 [0.83962264 0.83962264 0.83018868]
on test set
0.8333333333333334 [0.8375 0.8375 0.825 ]


In [9]:
params =  {'C': 1.0, 'degree': 3, 'kernel': 'rbf'}
model = SVC(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.84      1.00      0.91       200
         1.0       1.00      0.05      0.10        40

    accuracy                           0.84       240
   macro avg       0.92      0.53      0.50       240
weighted avg       0.87      0.84      0.78       240

